In [ ]:
import codecs
import networkx as nx
import os
import pandas as pd
import pickle
from os import listdir
from os.path import isfile, join
from networkx.drawing.nx_pydot import write_dot
from utils import printProgressBar

# column number Demonette
graph_1 = 3
graph_2 = 6
cat_1 = 8
cat_2 = 10
cstr_1 = 14
cstr_2 = 17
complexite = 19
orientation = 21
fichier_origine = 43

# column number Glawinette
lemma1 = 0
lemma2 = 1
cat1 = 2
cat2 = 3
origine_morpho = 4
origine_def = 5
BAP1 = 6
BAP2 = 7
BAPsize = 8
FAP1 = 9
FAP2 = 10
FAPsize = 11
radical = 12
FAPtype = 13

In [ ]:
folders_needed = ['DG-families', 'DG-graph', 'raw_csv_files']
for f in folders_needed:
    if not os.path.exists(f):
        os.makedirs(f)

In [ ]:
def FAPconverter(input_fap):
    return input_fap.replace('(.+)', 'X').replace('$', '').replace('^', '')

header = ''
glawi_dict = dict()
glawi_const_set = set()
with codecs.open('glawinette-series.csv', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        if line_num >= 1:
            elements = line.replace('\n','').replace(' ','').split('\t')
            construction12 = FAPconverter(elements[FAP1]) + '-' + FAPconverter(elements[FAP2])
            construction21 = FAPconverter(elements[FAP2]) + '-' + FAPconverter(elements[FAP1])
            glawi_const_set.add(construction12)
            glawi_const_set.add(construction21)
            glawi_dict[(elements[lemma1], elements[lemma2])] = construction12
            glawi_dict[(elements[lemma2], elements[lemma1])] = construction21
print(len(glawi_dict))

In [ ]:
glawi_const_set = list(glawi_const_set)
glawi_const_set.sort()

In [ ]:
input_dir = 'raw_csv_files'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
if '.gitignore' in input_files:
    input_files.remove('.gitignore')
input_files.sort()

input_file_code = dict()
input_file_code['converts'] = 'C'
input_file_code['demonette1'] = 'D'
input_file_code['mordan'] = 'M'

input_file_code['denomCONVX'] = 'Na'
input_file_code['denomPREFX'] = 'Nb'
input_file_code['denomXaire'] = 'Nc'
input_file_code['denomXal'] = 'Nd'
input_file_code['denomXique'] = 'Ne'
input_file_code['denomXSUF1'] = 'Nf'
input_file_code['denomXSUF2'] = 'Ng'
input_file_code['denomXSUF3'] = 'Nh'
input_file_code['denomXSUF4'] = 'Ni'
input_file_code['denomXSUF5'] = 'Nj'
input_file_code['denomXSUF6'] = 'Nk'

input_file_code['dimocXaie'] = 'Ma'
input_file_code['dimocXat'] = 'Mb'
input_file_code['dimocXet'] = 'Mc'
input_file_code['dimocXier'] = 'Md'
input_file_code['dimocXasmeXaste'] = 'Me'
input_file_code['dimocXite'] = 'Mf'
input_file_code['dimocXien'] = 'Mg'
input_file_code['dimocXisme'] = 'Mh'
input_file_code['dimocXiste'] = 'Mi'

input_file_code['derifantiX'] = 'Ra'
input_file_code['derifde1X'] = 'Rb'
input_file_code['derifenX'] = 'Rc'
input_file_code['derifinX'] = 'Rd'
input_file_code['derifQUANTX'] = 'Re'
input_file_code['derifreX'] = 'Rf'
input_file_code['deriftriX'] = 'Rg'
input_file_code['derifXable'] = 'Rh'
input_file_code['derifXiser'] = 'Ri'

# 1. Identify families of lexemes

In [ ]:
def find_representative(K):
    # a representative of a family: the root (no direct relation going in), or the node with the largest outdegree
    in_degree_dict = dict()  # only counting 'des2as' and 'as2des'
    for v in list(K.nodes):
        in_degree_dict[v] = 0
    for e in list(K.edges.data()):
        if not ('NA' in e[2]['label'] or 'indirect' in e[2]['label']):
            in_degree_dict[e[1]] += 1
    roots = list()
    for k in in_degree_dict:
        if in_degree_dict[k] == 0:
            roots.append(k)
    if len(roots) == 1:
        return roots[0].split('_')[0]
    elif len(roots) == 0:
        max_out_degree = 0
        selected_node = ''
        for n in list(K.nodes):
            if K.out_degree(n) > max_out_degree:
                max_out_degree = K.out_degree(n)
                selected_node = n
        return selected_node.split('_')[0]
    elif len(roots) > 1:
        max_out_degree = 0
        selected_root = roots[0]
        for r in roots:
            if K.out_degree(r) > max_out_degree:
                max_out_degree = K.out_degree(r)
                selected_root = r
        return selected_root.split('_')[0]
    
def category_shortening(cat):
    if cat != 'Num' and cat[0] == 'N':
        if cat[1] == 'p':  # nom propre
            return 'Np'
        return 'N'  # nom
    return cat

Create arc between two lexemes

In [ ]:
header = ''
G = nx.DiGraph()
number_of_pairs = 0
number_of_unique_pairs = 0
for input_file in input_files:
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num == 0:
                header = line.replace('\n','')
            elif line_num >= 2:
                elements = line.replace('\n','').replace(' ','').split('\t')
                v1 = elements[graph_1] + '_' + elements[cat_1]
                v2 = elements[graph_2] + '_' + elements[cat_2]
                if G.has_edge(v1, v2) or G.has_edge(v2, v1):
                    continue
                G.add_node(v1, label=category_shortening(elements[cat_1]))
                G.add_node(v2, label=category_shortening(elements[cat_2]))
                if elements[orientation] == 'as2de' or elements[orientation] == 'as2des':
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        G.add_edge(v1, v2, label=elements[cstr_1] + '-' + elements[cstr_2]\
                                   + '$' + glawi_dict.get((elements[graph_1], elements[graph_2])))
                    else:
                        G.add_edge(v1, v2, label=elements[cstr_1] + '-' + elements[cstr_2])
                elif elements[orientation] == 'de2as' or elements[orientation] == 'des2as':
                    if (elements[graph_2], elements[graph_1]) in glawi_dict.keys():
                        G.add_edge(v2, v1, label=elements[cstr_2] + '-' + elements[cstr_1]\
                                   + '$' + glawi_dict.get((elements[graph_2], elements[graph_1])))
                    else:
                        G.add_edge(v2, v1, label=elements[cstr_2] + '-' + elements[cstr_1])
                else:
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        G.add_edge(v1, v2, label=elements[cstr_1] + '-' + elements[cstr_2] + '_' + elements[orientation]\
                                   + '$' + glawi_dict.get((elements[graph_1], elements[graph_2])) +  '_' + elements[orientation])
                        G.add_edge(v2, v1, label=elements[cstr_2] + '-' + elements[cstr_1] + '_' + elements[orientation]\
                                   + '$' + glawi_dict.get((elements[graph_2], elements[graph_1])) +  '_' + elements[orientation])
                    else:
                        G.add_edge(v1, v2, label=elements[cstr_1] + '-' + elements[cstr_2] + '_' + elements[orientation])
                        G.add_edge(v2, v1, label=elements[cstr_2] + '-' + elements[cstr_1] + '_' + elements[orientation])

A connected component is regarded as a family

In [ ]:
conn_comps = list(nx.weakly_connected_components(G))
number_of_families = len(conn_comps)
print(number_of_families, 'families')

Comparing every pair of families to detect isomorphy groups

O(n^2)

In [ ]:
checked = list()
H = nx.Graph() # graph of graphs
for fam1 in range(0, number_of_families):
    printProgressBar(fam1 + 1, number_of_families, prefix = 'Progress:', suffix = 'complete', length = 50, decimals = 2)
    if fam1 in checked:
        continue
    H.add_node(fam1)
    for fam2 in range(fam1 + 1, number_of_families):
        if fam2 in checked:
            continue
        G1 = nx.subgraph(G, conn_comps[fam1])
        G2 = nx.subgraph(G, conn_comps[fam2])
        if nx.is_isomorphic(G1, G2, node_match=lambda v1,v2: v1['label'] == v2['label'],\
                            edge_match=lambda e1,e2: e1['label'] == e2['label']):
            checked.append(fam2)
            H.add_edge(fam1, fam2)

In [ ]:
H_conn_comps = [c for c in sorted(nx.connected_components(H), key=len, reverse=False)]
print(len(H_conn_comps), 'isomorphy groups')

Generate csv file for each family to folder "DG-families", and

Generate "DG_summary_of_families.txt" that records the list of lexemes for each family

In [ ]:
output_folder = 'DG-families'
lexeme_dict = dict()
f_summary = codecs.open('DG_summary_of_families.txt', 'w+', encoding='utf-8')
f_summary.write('family_id\tnumber_of_lexemes\tlexemes\n')
for fam_fam_id, fam_fam in enumerate(H_conn_comps):
    for fam_id, fam in enumerate(fam_fam):
        representative = find_representative(nx.subgraph(G, conn_comps[fam]))
        if len(fam_fam) == 1:
            family_title = 'F' + str(fam_fam_id).rjust(5, '0')
        else:
            family_title = 'F' + str(fam_fam_id).rjust(5, '0') + '-' + str(fam_id).rjust(len(str(len(fam_fam)-1)), '0')
        f_summary.write(family_title + '\t'\
                       + str(len(conn_comps[fam])) + '\t' + str(conn_comps[fam]) + '\n')
        for lexeme in conn_comps[fam]:
            filename = family_title + ' ' + representative + '.txt'
            lexeme_dict[lexeme] = filename
            f_out = codecs.open(join(output_folder, filename), 'w+', encoding='utf-8')
            f_out.write(header + '\tfichier_origine' + '\n\n')
            f_out.close()
f_summary.close()

In [ ]:
for input_file in input_files:
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num < 2:
                continue
            elements = line.replace(' ','').split('\t')
            lexeme1 = elements[graph_1] + '_' + elements[cat_1]
            if elements[complexite] not in ['simple', 'complexe', 'motiv-form', 'motiv-sem', 'accidentel']:
                print('warning ', input_file)
            output_filename = lexeme_dict[lexeme1]
            f_out = codecs.open(join(output_folder, output_filename), 'a+', encoding='utf-8')
            f_out.write(line.strip('\n') + '\t' + input_file_code.get(input_file.split('-')[0], '?') + '\n')
            f_out.close()

# 2. Generate graphs (.dot files)

In [ ]:
def category_shortening(cat):
    if cat != 'Num' and cat[0] == 'N':
        if cat[1] == 'p':  # nom propre
            return 'Np'
        return 'N'  # nom
    return cat

def edge_writer(L):
    ret_str = ''
    edges = L.edges(data=True)
    for e in edges:
        cat0 = category_shortening(e[0].split('_')[1])
        cat1 = category_shortening(e[1].split('_')[1])
        label_demon = e[2]['label'].split('\n')[0]
        if e[2]['style'] == 'dotted': 
            ret_str += cat0 + ' -- ' + label_demon.split(': ')[1] + ' -- ' + cat1 + '; '
        elif e[2]['style'] == 'dashed':
            ret_str += cat0 + ' -? ' + label_demon.split(': ')[1] + ' -? ' + cat1 + '; '
        else:
            ret_str += cat0 + ' -> ' + label_demon.split(': ')[1] + ' -> ' + cat1 + '; '
        if '\n' in e[2]['label']:
            label_glawi = e[2]['label'].split('\n')[1]
            if e[2]['style'] == 'dotted': 
                ret_str += cat0 + ' -- ' + label_glawi.split(': ')[1] + ' -- ' + cat1 + '; '
            elif e[2]['style'] == 'dashed':
                ret_str += cat0 + ' -? ' + label_glawi.split(': ')[1] + ' -? ' + cat1 + '; '
            else:
                ret_str += cat0 + ' -> ' + label_glawi.split(': ')[1] + ' -> ' + cat1 + '; '
    return ret_str

Generate .dot file for each family, and "DG_summary_of_groups.txt" that records the list of edges for each isomorphy group

In [ ]:
f_summary = codecs.open('DG_summary_of_groups.txt', 'w+', encoding='utf-8')
f_summary.write('group id\tnumber of lexemes\tnumber of edges\tnumber of families\tedges\tfamilies\n')
number_of_edges = []
number_of_families = []
words = ''
group_prec = ''
family_count = 0
L = nx.DiGraph()
family_dir = 'DG-families'
family_files = [f for f in listdir(family_dir) if isfile(join(family_dir, f))]
if '.gitignore' in family_files:
    family_files.remove('.gitignore')
for family_file in family_files:
    group_id = family_file.split(' ')[0].split('-')[0]
    if group_id != group_prec and group_prec != '':
        f_summary.write(group_prec + '\t' + str(len(L)) + '\t' + str(L.size()) + '\t' + str(family_count) + '\t')
        f_summary.write(edge_writer(L)[:-2])
        f_summary.write('\t' + words[:-2] + '\n')
        family_count = 0
        words = ''
    family_count += 1
    group_prec = group_id
    L = nx.DiGraph()
    with codecs.open(join(family_dir, family_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                elements = line.replace('\n','').replace(' ','').split('\t')
                v1 = elements[graph_1] + '_' + elements[cat_1]
                v2 = elements[graph_2] + '_' + elements[cat_2]
                if L.has_edge(v1, v2) or L.has_edge(v2, v1):
                    continue
                L.add_node(v1, label=elements[graph_1] + '\n' + category_shortening(elements[cat_1]))
                L.add_node(v2, label=elements[graph_2] + '\n' + category_shortening(elements[cat_2]))
                if elements[orientation] == 'as2de' or elements[orientation] == 'as2des':
                    edge_type = elements[fichier_origine] + ': ' + elements[cstr_1] + '-' + elements[cstr_2]
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        L.add_edge(v1, v2, label=edge_type + '\nG: ' + glawi_dict.get((elements[graph_1], elements[graph_2])), style='')
                    else:
                        L.add_edge(v1, v2, label=edge_type, style='')
                elif elements[orientation] == 'de2as' or elements[orientation] == 'des2as':
                    edge_type = elements[fichier_origine] + ': ' + elements[cstr_2] + '-' + elements[cstr_1]
                    if (elements[graph_2], elements[graph_1]) in glawi_dict.keys():
                        L.add_edge(v2, v1, label=edge_type + '\nG: ' + glawi_dict.get((elements[graph_2], elements[graph_1])), style='')
                    else:
                        L.add_edge(v2, v1, label=edge_type, style='')
                elif elements[orientation] == 'indirect':
                    sorted_lex = sorted([v1, v2])
                    edge_type = elements[fichier_origine] + ': ' + elements[cstr_1] + '-' + elements[cstr_2]
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        L.add_edge(v1, v2, dir='none', style='dotted', \
                                   label=edge_type + '\nG: ' + glawi_dict.get((elements[graph_1], elements[graph_2])))
                    else:
                        L.add_edge(sorted_lex[0], sorted_lex[1], dir='none', style='dotted', label=edge_type)
                elif elements[orientation] == 'NA':
                    sorted_lex = sorted([v1, v2])
                    edge_type = elements[fichier_origine] + ': ' + elements[cstr_1] + '-' + elements[cstr_2]
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        L.add_edge(v1, v2, dir='none', style='dashed',\
                                   label=edge_type + '\nG: ' + glawi_dict.get((elements[graph_1], elements[graph_2])))
                    else:
                        L.add_edge(sorted_lex[0], sorted_lex[1], dir='none', style='dashed', label=edge_type)
    words += str(list(L.nodes())) + '; '
    write_dot(L, join('DG-graph', family_file.replace('.txt','.dot')))
    print(family_file.split(' ')[0], end='\r')
f_summary.write(group_prec + '\t' + str(len(L)) + '\t' + str(L.size()) + '\t' + str(family_count) + '\t')
f_summary.write(edge_writer(L)[:-2])
f_summary.write('\t' + words[:-2] + '\n')  
f_summary.close()